In [34]:
import matplotlib.pyplot as plt
import cv2, random
import albumentations as A

from albumentations.pytorch import ToTensorV2
from data_loader.dataloader import *

### 1. DataLoader

- transform 반드시 넣어줘야한다.

#### 1 ) transform에 normalize가 없으면 파손 이미지를 255로 나누어준다.

In [35]:
tmp_transform = A.Compose([A.Resize(512, 512), ToTensorV2(transpose_mask=True)])

In [36]:
tmp_data = Custom_dataset(data_dir='./dataset/dent/train/', transform=tmp_transform)

In [40]:
tmp_data[0]['input'].max(), tmp_data[0]['input'].min()

(tensor(0.9451), tensor(0.0235))

In [41]:
tmp_data[0]['label'].max(), tmp_data[0]['label'].min()

(tensor(1.), tensor(0.))

#### 2 ) transform에 normalize가 있으면 Normalize 진행한다.

In [42]:
tmp_transform = A.Compose([A.Resize(512, 512), A.Normalize(), ToTensorV2(transpose_mask=True)])

In [43]:
tmp_data = Custom_dataset(data_dir='./dataset/dent/train/', transform=tmp_transform)

- Problem 1. 혹시 파손 이미지 값이 -1 ~ 1 사이의 값이 아니어서 학습이 안되는 것은 아닐까?

In [44]:
tmp_data[0]['input'].max(), tmp_data[0]['input'].min()

(tensor(2.3960), tensor(-1.9809))

In [45]:
tmp_data[0]['label'].max(), tmp_data[0]['label'].min()

(tensor(1.), tensor(0.))

### 2. Metric

#### 1 ) IOU score

In [1]:
from model.metric import *

In [25]:
t_zero = torch.zeros(size=(32, 3, 128, 128))
t_ones = torch.ones(size=(32, 3, 128, 128))
t_randn = torch.randn(size=(32, 3, 128, 128))
t_prob = (torch.sigmoid(t_randn) > 0.5).float()

- 모든 실제 마스크 영역이 파손 / 모든 예측 마스크 영역이 정상

In [26]:
IOUscore(None, t_zero, t_ones)

0.0

- 모든 실제 마스크 영역이 파손 / 모든 예측 마스크 영역이 파손

In [31]:
IOUscore(None, t_ones, t_ones)

1.0

- 모든 실제 마스크 영역이 정상 / 모든 예측 마스크 영역이 정상

In [27]:
IOUscore(None, t_zero, t_zero)

nan

- 모든 실제 마스크 영역이 정상 / 모든 예측 마스크 영역이 파손

In [28]:
IOUscore(None, t_ones, t_zero)

0.0

- 모든 실제 마스크 영역이 파손 / 예측 마스크 영역이 랜덤

In [29]:
IOUscore(None, t_prob, t_ones)

0.4999014536539714

- 모든 실제 마스크 영역이 정상 / 예측 마스크 영역이 랜덤

In [30]:
IOUscore(None, t_prob, t_zero)

0.0

#### 2 ) Pixel Accuracy

In [33]:
PixelAccuracy(None,t_prob, t_ones)

tensor(49.9901)

In [34]:
PixelAccuracy(None, t_prob, t_zero)

tensor(50.0099)